# Imports

In [1]:
import random
from z3 import *
import time

## Dummy Data Generation

In [2]:
# --- Configuration for Generation ---
NUM_COMPONENTS = 40  
NUM_ECUS = 10          
RANDOM_SEED = 42 

# Definitions 
COMPONENTS = [f'C_{i}' for i in range(1, NUM_COMPONENTS + 1)]
ECUS = [f'E_{j}' for j in range(1, NUM_ECUS + 1)]
M = len(COMPONENTS)

# --- Percentages for Generation ---
# Supply (ECUs) - Counts
COUNT_HPC = 3
COUNT_SAFETY = 4
COUNT_GATEWAY = 3

# Demand (Components) - Weights
WEIGHT_PERCEPTION = 0.25
WEIGHT_CONTROL = 0.25
WEIGHT_BODY = 0.30
WEIGHT_INFOTAINMENT = 0.2  

# ASIL Mapping 
ASIL_MAP = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}

def generate_robust_data(cnt_hpc, cnt_safety, cnt_gateway, w_perception, w_control, w_body, w_infotainment):
    """Generates synthetic data"""
    
    if RANDOM_SEED is not None:
        random.seed(RANDOM_SEED)
    
    # --- 1. Define ECU Profiles (The Supply) ---
    # Create list based on counts
    ecu_types_list = ['HPC']*cnt_hpc + ['SAFETY']*cnt_safety + ['GATEWAY']*cnt_gateway
    
    # Fill remainder if NUM_ECUS > sum of counts
    while len(ecu_types_list) < NUM_ECUS:
        ecu_types_list.append(random.choice(['HPC', 'SAFETY', 'GATEWAY']))
    
    random.shuffle(ecu_types_list) 
    
    # Initialize Dictionaries with High-Fidelity Units
    max_cpu, max_ram, max_rom, max_gpu = {}, {}, {}, {}
    base_cost = {}
    ecu_asil = {}
    ecu_caps = {}
    ecu_tiers = {}
    
    for idx, tier in enumerate(ecu_types_list):
        j = ECUS[idx] 
        ecu_tiers[j] = tier
        
        if tier == 'HPC':
            # REALISTIC SPECS: Orin/Snapdragon Class
            max_cpu[j] = 250000.0  # 250k DMIPS
            max_gpu[j] = 500.0     # 500 TOPS (AI)
            max_ram[j] = 65536.0   # 64 GB RAM
            max_rom[j] = 256000.0  # 256 GB Storage
            base_cost[j] = 200.0
            ecu_asil[j] = 'B' 
            ecu_caps[j] = ['HW_ETH']
            
        elif tier == 'SAFETY':
            # REALISTIC SPECS: Aurix TC4x Class
            max_cpu[j] = 16000.0   # 16k DMIPS
            max_gpu[j] = 0.0       # No AI Accel
            max_ram[j] = 24.0      # 24 MB SRAM
            max_rom[j] = 64.0      # 64 MB Flash
            base_cost[j] = 80.0
            ecu_asil[j] = 'D' 
            ecu_caps[j] = ['HW_HSM', 'HW_FLEX', 'HW_CANFD']
            
        else: # GATEWAY
            # REALISTIC SPECS: S32G Class
            max_cpu[j] = 8000.0    # 8k DMIPS
            max_gpu[j] = 0.0
            max_ram[j] = 8.0       # 8 MB SRAM
            max_rom[j] = 16.0      # 16 MB Flash
            base_cost[j] = 30.0
            ecu_asil[j] = 'B'
            ecu_caps[j] = ['HW_HSM', 'HW_CANFD','HW_ETH']

    # --- 2. Define Component Profiles (The Demand) ---
    req_cpu, req_ram, req_rom, req_gpu = {}, {}, {}, {}
    req_asil, req_hw = {}, {}
    comp_types = {} 
    
    for i in COMPONENTS:
        # Use the weights passed as arguments
        ctype = random.choices(
            ['PERCEPTION', 'CONTROL', 'BODY', "INFOTAINMENT"], 
            weights=[w_perception, w_control, w_body, w_infotainment] 
        )[0]
        comp_types[i] = ctype
        
        if ctype == 'PERCEPTION':
            # High Load AI Task
            req_cpu[i] = random.uniform(15000, 40000) # 15k-40k DMIPS
            req_gpu[i] = random.uniform(20, 150)       # 20-80 TOPS
            req_ram[i] = random.uniform(2048*2, 8192*2)   # 2-8 GB RAM
            req_rom[i] = random.uniform(1000, 5000)   # 1-5 GB Storage
            req_asil[i] = random.choice(['QM', 'B'])
            req_hw[i] = ['HW_ETH']
            
        elif ctype == 'CONTROL':
            # Critical Real-Time Task
            req_cpu[i] = random.uniform(500, 3000)    # 500-3k DMIPS
            req_gpu[i] = 0.0
            req_ram[i] = random.uniform(1, 5)         # 1-5 MB
            req_rom[i] = random.uniform(2, 10)        # 2-10 MB
            req_asil[i] = 'D' 
            req_hw[i] = ['HW_FLEX'] 

        elif ctype == 'INFOTAINMENT':
            # Medium Load Task
            req_cpu[i] = random.uniform(5000, 20000)  # 2k-10k DMIPS
            req_gpu[i] = random.uniform(10, 80)        # 1-10 TOPS
            req_ram[i] = random.uniform(2048*2, 8192*2)    # 512 MB - 2 GB
            req_rom[i] = random.uniform(1000, 2000)    # 500 MB - 2 GB
            req_asil[i] = random.choice(['QM', 'A'])
            req_hw[i] = ['HW_ETH']
            
        else: # BODY
            # Simple Logic
            req_cpu[i] = random.uniform(50, 300)      # 50-300 DMIPS
            req_gpu[i] = 0.0
            req_ram[i] = random.uniform(0.1, 1.0)     # <1 MB
            req_rom[i] = random.uniform(0.5, 2.0)     # <2 MB
            req_asil[i] = random.choice(['QM', 'A'])
            req_hw[i] = [] 

    # --- 3. Comm Matrix ---
    comm_vol = {}
    for i in COMPONENTS:
        for k in COMPONENTS:
            if i < k and random.random() < 0.3: # 10% chance of communication 
                comm_vol[(i, k)] = random.randint(5, 50)
                comm_vol[(k, i)] = comm_vol[(i, k)]

    # --- 4. Critical Pairs ---
    critical_pairs = []
    safety_comps = [c for c, a in req_asil.items() if a == 'D']
    if len(safety_comps) >= 2:
        c1, c2 = random.sample(safety_comps, 2)
        critical_pairs.append((c1, c2))

    return (max_cpu, max_ram, max_rom, max_gpu, base_cost, ecu_caps, ecu_asil, ecu_tiers,
            req_cpu, req_ram, req_rom, req_gpu, req_asil, req_hw, comp_types,
            comm_vol, critical_pairs)

In [3]:
# --- EXECUTE GENERATION ---
# 1. Call the function with the correct parameters
# 2. Unpack ALL 17 return values including the new ROM/GPU data
(MAX_CPU, MAX_RAM, MAX_ROM, MAX_GPU, ECU_BASE_COST, ECU_HW_CAPABILITIES, ECU_ASIL_CAPABILITY, ECU_TIERS,
 REQ_CPU, REQ_RAM, REQ_ROM, REQ_GPU, REQ_ASIL, REQ_HW, COMP_TYPES,
 COMM_VOLUME, CRITICAL_PAIRS_TO_SEPARATE) = generate_robust_data(
    COUNT_HPC, COUNT_SAFETY, COUNT_GATEWAY, 
    WEIGHT_PERCEPTION, WEIGHT_CONTROL, WEIGHT_BODY, WEIGHT_INFOTAINMENT
)

# --- PRE-SOLVE DIAGNOSTICS ---
print(f"\n{'='*60}")
print(f"      SCENARIO SUMMARY (Seed: {RANDOM_SEED})")
print(f"{'='*60}")

# Check 1: CPU Supply vs Demand (DMIPS)
total_cpu_supply = sum(MAX_CPU.values())
total_cpu_demand = sum(REQ_CPU.values())
print(f"CPU Supply (DMIPS): {total_cpu_supply:,.0f} | Demand: {total_cpu_demand:,.0f}")
if total_cpu_demand > total_cpu_supply:
    print("FAILURE: Global CPU Shortage")

# Check 2: GPU Supply vs Demand (TOPS)
# Now we compare actual TOPS, not just CPU power of GPU nodes.
total_gpu_supply = sum(MAX_GPU.values())
total_gpu_demand = sum(REQ_GPU.values())
print(f"GPU Supply (TOPS):  {total_gpu_supply:,.0f} | Demand: {total_gpu_demand:,.0f}")

if total_gpu_demand > total_gpu_supply:
    print("FAILURE: Not enough AI Acceleration (TOPS)!")
elif total_gpu_demand > total_gpu_supply * 0.9:
    print("WARNING: GPU Saturation > 90%")
else:
    print("GPU Supply Healthy")

# Check 3: ASIL D Constraints
safety_ecus = [e for e, lvl in ECU_ASIL_CAPABILITY.items() if lvl == 'D']
safety_comps = [c for c, lvl in REQ_ASIL.items() if lvl == 'D']
saf_supply = sum(MAX_CPU[e] for e in safety_ecus)
saf_demand = sum(REQ_CPU[c] for c in safety_comps)

print(f"Safety Nodes: {len(safety_ecus)} ECUs | Critical Tasks: {len(safety_comps)}")
if saf_demand > saf_supply:
     print("FAILURE: Safety Tier CPU Overload")
else:
     print("Safety Tier Supply Healthy")

# --- DETAILED DATA INSPECTOR ---
print(f"\n{'='*80}")
print(f"      DETAILED ECU DATA (Supply)")
print(f"{'='*80}")
# Adjusted formatting for larger numbers
print(f"{'ECU':<6} | {'TIER':<8} | {'ASIL':<5} | {'CPU(DMIPS)':<11} | {'GPU(TOPS)':<10} | {'RAM(MB)':<9} | {'ROM(MB)':<9} | {'COST':<5}")
print("-" * 80)
for j in ECUS:
    print(f"{j:<6} | {ECU_TIERS[j]:<8} | {ECU_ASIL_CAPABILITY[j]:<5} | {MAX_CPU[j]:<11.0f} | {MAX_GPU[j]:<10.0f} | {MAX_RAM[j]:<9.0f} | {MAX_ROM[j]:<9.0f} | {ECU_BASE_COST[j]:<5.0f}")

print(f"\n{'='*80}")
print(f"      DETAILED COMPONENT DATA (Demand)")
print(f"{'='*80}")
print(f"{'COMP':<6} | {'TYPE':<12} | {'ASIL':<5} | {'CPU':<7} | {'GPU':<5} | {'RAM':<6} | {'ROM':<6} | {'REQ HW'}")
print("-" * 80)
# Showing first 15 components to keep log readable, remove slice [:15] to see all
for i in COMPONENTS: 
    hw = ",".join(REQ_HW[i]) if REQ_HW[i] else "-"
    # Format floats to 1 decimal place
    print(f"{i:<6} | {COMP_TYPES[i]:<12} | {REQ_ASIL[i]:<5} | {REQ_CPU[i]:<7.0f} | {REQ_GPU[i]:<5.1f} | {REQ_RAM[i]:<6.1f} | {REQ_ROM[i]:<6.1f} | {hw}")

print("\nData generation complete.")


      SCENARIO SUMMARY (Seed: 42)
CPU Supply (DMIPS): 838,000 | Demand: 350,190
GPU Supply (TOPS):  1,500 | Demand: 1,028
GPU Supply Healthy
Safety Nodes: 4 ECUs | Critical Tasks: 11
Safety Tier Supply Healthy

      DETAILED ECU DATA (Supply)
ECU    | TIER     | ASIL  | CPU(DMIPS)  | GPU(TOPS)  | RAM(MB)   | ROM(MB)   | COST 
--------------------------------------------------------------------------------
E_1    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 30   
E_2    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_3    | HPC      | B     | 250000      | 500        | 65536     | 256000    | 200  
E_4    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 30   
E_5    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_6    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_7    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 

## Z3 Solver

In [4]:
# --- 1. Preprocessing (Reals -> Integers) ---
SCALE = 10 

def to_z3_int(val):
    return int(val * SCALE)

INT_MAX_CPU = {k: to_z3_int(v) for k, v in MAX_CPU.items()}
INT_MAX_RAM = {k: to_z3_int(v) for k, v in MAX_RAM.items()}
INT_MAX_ROM = {k: to_z3_int(v) for k, v in MAX_ROM.items()}
INT_MAX_GPU = {k: to_z3_int(v) for k, v in MAX_GPU.items()}
INT_REQ_CPU = [to_z3_int(REQ_CPU[c]) for c in COMPONENTS]
INT_REQ_RAM = [to_z3_int(REQ_RAM[c]) for c in COMPONENTS]
INT_REQ_ROM = [to_z3_int(REQ_ROM[c]) for c in COMPONENTS]
INT_REQ_GPU = [to_z3_int(REQ_GPU[c]) for c in COMPONENTS]

# --- 2. MODEL KURULUMU ---
opt = Optimize()
opt.set(priority="pareto")
# Decision Variable placement[i] = j
placement = [Int(f"p_{i}") for i in range(NUM_COMPONENTS)]
# Domain Constraint: 0 <= placement < NUM_ECUS
for p in placement:
    opt.add(p >= 0, p < NUM_ECUS)

# --- 3. HARD CONSTRAINTS ---
# Source Constraints 
for j in range(NUM_ECUS):
    # if placement[i] == j ise 1, otherwise 0
    assigned_flags = [If(placement[i] == j, 1, 0) for i in range(NUM_COMPONENTS)]
    
    # CPU Load
    load_cpu = Sum([assigned_flags[i] * INT_REQ_CPU[i] for i in range(NUM_COMPONENTS)])
    opt.add(load_cpu <= INT_MAX_CPU[ECUS[j]])
    # RAM Load
    load_ram = Sum([assigned_flags[i] * INT_REQ_RAM[i] for i in range(NUM_COMPONENTS)])
    opt.add(load_ram <= INT_MAX_RAM[ECUS[j]])
    # ROM Load
    load_rom = Sum([assigned_flags[i] * INT_REQ_ROM[i] for i in range(NUM_COMPONENTS)])
    opt.add(load_rom <= INT_MAX_ROM[ECUS[j]])
    # GPU Load
    load_gpu = Sum([assigned_flags[i] * INT_REQ_GPU[i] for i in range(NUM_COMPONENTS)])
    opt.add(load_gpu <= INT_MAX_GPU[ECUS[j]])

# HW Compatibility & ASIL
for i, c_name in enumerate(COMPONENTS):
    req_hw_set = set(REQ_HW[c_name])
    req_asil_val = ASIL_MAP[REQ_ASIL[c_name]]
    for j, e_name in enumerate(ECUS):
        # 1. HW Check
        if not req_hw_set.issubset(set(ECU_HW_CAPABILITIES[e_name])):
            opt.add(placement[i] != j)
        # 2. ASIL Check
        if req_asil_val > ASIL_MAP[ECU_ASIL_CAPABILITY[e_name]]:
            opt.add(placement[i] != j)

# Safety Separation (Critical Pairs)
comp_name_to_idx = {name: i for i, name in enumerate(COMPONENTS)}
for (c1, c2) in CRITICAL_PAIRS_TO_SEPARATE:
    idx1 = comp_name_to_idx[c1]
    idx2 = comp_name_to_idx[c2]
    opt.add(placement[idx1] != placement[idx2])

In [5]:
# --- 4. OBJECTIVES---

comm_cost = Int('comm_cost')
sys_cost = Int('sys_cost')
balance_cost = Int('balance_cost')

# -- Comm Cost Terms --
comm_terms = []
for (c1, c2), vol in COMM_VOLUME.items():
    i1 = comp_name_to_idx[c1]
    i2 = comp_name_to_idx[c2]
    term = If(placement[i1] != placement[i2], int(vol), 0)
    comm_terms.append(term)
opt.add(comm_cost == Sum(comm_terms))

# -- Balance Cost Terms --
balance_cost = Int('balance_cost')
balance_terms = []

# 1. Define Resource Data and Weights
#Adjust weights to handle scale differences.
resource_map = {
    "CPU": (INT_REQ_CPU, 1),   # Assuming CPU values are small (e.g. cores), weight up
    "GPU": (INT_REQ_GPU, 100),   # Assuming GPU values are small
    "RAM": (INT_REQ_RAM, 1000),    # Assuming RAM values are large (MB), weight down
    "ROM": (INT_REQ_ROM, 1)     # Assuming ROM values are large
}

for r_name, (req_list, w_r) in resource_map.items():
    
    # Calculate the ideal constant average for this resource
    total_res_req = sum(req_list)
    avg_res_int = total_res_req // NUM_ECUS 
    
    # Iterate through each ECU to calculate deviation for this resource
    for j in range(NUM_ECUS):
        d_j_r = Int(f'dev_{r_name}_{j}') # Variable for absolute deviation
        

        load_on_j_r = Sum([If(placement[i] == j, req_list[i], 0) for i in range(NUM_COMPONENTS)])
        # Linearization Constraints: d >= |Load - Avg|
        # 1. d >= Load - Avg
        opt.add(d_j_r >= (load_on_j_r - avg_res_int))
        # 2. d >= Avg - Load
        opt.add(d_j_r >= (avg_res_int - load_on_j_r))
        # 3. Non-negativity
        opt.add(d_j_r >= 0)
        
        balance_terms.append(w_r * d_j_r)

# Sum all resource deviations
opt.add(balance_cost == Sum(balance_terms))

# -- System Cost Terms --
cost_terms = []
for j in range(NUM_ECUS):
    load_on_j = Sum([If(placement[i] == j, INT_REQ_CPU[i], 0) for i in range(NUM_COMPONENTS)])
    cost_terms.append(If(load_on_j > 0, int(ECU_BASE_COST[ECUS[j]]), 0))
opt.add(sys_cost == Sum(cost_terms))


h1 = opt.minimize(comm_cost)
h2 = opt.minimize(sys_cost)
h3 = opt.minimize(balance_cost)



In [6]:
# --- 5. SOLVE ---
print("Z3 30 seconds...")
opt.set("timeout", 30000)

status = opt.check()

if status == sat or status == unknown:
    m = opt.model()
    print("Solution Found!")
    
    #print(f"Comm Cost: {m.eval(comm_cost)}")
    #print(f"Sys Cost:  {m.eval(sys_cost)}")
    

    print("\n--- Assignment ---")
    assignments = {}
    for i in range(NUM_COMPONENTS):
        ecu_idx = m.eval(placement[i], model_completion=True).as_long()
        assignments.setdefault(ECUS[ecu_idx], []).append(COMPONENTS[i])
    
    for ecu, comps in assignments.items():
        print(f"{ecu}: {comps}")
    
    print("\n=== ECU Usage and Resource Load ===")
    
    for j in range(NUM_ECUS):
        ecu_name = ECUS[j]
        
        # FInd assigned components
        assigned_indices = [i for i in range(NUM_COMPONENTS) 
                            if m.eval(placement[i]).as_long() == j]
        
        if not assigned_indices:
            continue
            

        used_cpu = sum(REQ_CPU[COMPONENTS[i]] for i in assigned_indices)
        used_ram = sum(REQ_RAM[COMPONENTS[i]] for i in assigned_indices)
        used_rom = sum(REQ_ROM[COMPONENTS[i]] for i in assigned_indices)
        used_gpu = sum(REQ_GPU[COMPONENTS[i]] for i in assigned_indices)
        cap_cpu = MAX_CPU[ecu_name]
        cap_ram = MAX_RAM[ecu_name]
        cap_rom = MAX_ROM[ecu_name]
        cap_gpu = MAX_GPU[ecu_name]
        
        # 4. Calculate Percentages
        perc_cpu = (used_cpu / cap_cpu * 100) if cap_cpu > 0 else 0
        perc_ram = (used_ram / cap_ram * 100) if cap_ram > 0 else 0
        perc_rom = (used_rom / cap_rom * 100) if cap_rom > 0 else 0
        perc_gpu = (used_gpu / cap_gpu * 100) if cap_gpu > 0 else 0
        
        output_str = f"{ecu_name} (Used):\n"
        output_str += f"CPU Load: {used_cpu:.1f} / {cap_cpu:.1f} ({perc_cpu:.1f}%)\n"
        output_str += f"RAM Load: {used_ram:.1f} / {cap_ram:.1f} ({perc_ram:.1f}%)\n"
        output_str += f"ROM Load: {used_rom:.1f} / {cap_rom:.1f} ({perc_rom:.1f}%)\n"
        if cap_gpu > 0:
            output_str += f"GPU Load: {used_gpu:.1f} / {cap_gpu:.1f} ({perc_gpu:.1f}%)\n"
            
        print(output_str)


elif status == unsat:
    print("UNSAT")
else:
    print("Error occurred.")

Z3 30 seconds...
Solution Found!

--- Assignment ---
E_1: ['C_1', 'C_2', 'C_3', 'C_6', 'C_10', 'C_17', 'C_18', 'C_20', 'C_23', 'C_28']
E_5: ['C_4', 'C_16', 'C_29', 'C_39']
E_10: ['C_5', 'C_9', 'C_15', 'C_22', 'C_40']
E_3: ['C_7', 'C_8', 'C_13', 'C_14', 'C_21', 'C_26', 'C_33', 'C_35']
E_2: ['C_11', 'C_12', 'C_24', 'C_27', 'C_30', 'C_32', 'C_34', 'C_37']
E_9: ['C_19', 'C_25', 'C_31', 'C_36', 'C_38']

=== ECU Usage and Resource Load ===
E_1 (Used):
CPU Load: 1949.0 / 8000.0 (24.4%)
RAM Load: 4.3 / 8.0 (53.9%)
ROM Load: 9.9 / 16.0 (61.9%)

E_2 (Used):
CPU Load: 15106.3 / 16000.0 (94.4%)
RAM Load: 19.7 / 24.0 (82.1%)
ROM Load: 40.8 / 64.0 (63.7%)

E_3 (Used):
CPU Load: 127662.3 / 250000.0 (51.1%)
RAM Load: 65488.1 / 65536.0 (99.9%)
ROM Load: 12928.9 / 256000.0 (5.1%)
GPU Load: 407.1 / 500.0 (81.4%)

E_5 (Used):
CPU Load: 7379.3 / 16000.0 (46.1%)
RAM Load: 12.3 / 24.0 (51.4%)
ROM Load: 23.7 / 64.0 (37.0%)

E_9 (Used):
CPU Load: 103228.6 / 250000.0 (41.3%)
RAM Load: 61071.6 / 65536.0 (93.2%)
